In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn import (
    model_selection,
    preprocessing,
    dummy,
    metrics,
    ensemble,
    tree,
    neighbors,
    pipeline,
    compose,
    linear_model,
    svm,
)
import xgboost as xgb

ARTIFACTS_DIR = "../models/artifacts/"
MODEL_DIR = "../models/"

In [2]:
CAT_COLS = [
    "gender",
    "SeniorCitizen",
    "Partner",
    "Dependents",
    "PhoneService",
    "MultipleLines",
    "InternetService",
    "OnlineSecurity",
    "OnlineBackup",
    "DeviceProtection",
    "TechSupport",
    "StreamingTV",
    "StreamingMovies",
    "Contract",
    "PaperlessBilling",
    "PaymentMethod",
]
NUM_COLS = ["tenure", "MonthlyCharges", "TotalCharges"]
TARGET = ["Churn"]
SEED = 69420

df = pd.read_csv(
    "../data/WA_Fn-UseC_-Telco-Customer-Churn.csv", usecols=CAT_COLS + NUM_COLS + TARGET
)
df.shape

(7043, 20)

In [3]:
CAT_COLS_OHE = ["PaymentMethod", "Contract", "InternetService"]
CAT_COLS_LE = list(set(CAT_COLS) - set(CAT_COLS_OHE))

In [4]:
df.TotalCharges = df.TotalCharges.replace(to_replace=" ", value="0")


def custom_combiner(feature, category):
    return str(feature) + "_" + type(category).__name__ + "_" + str(category)


scaler = preprocessing.StandardScaler().set_output(transform="pandas")
encoder_ohe = preprocessing.OneHotEncoder(feature_name_combiner=custom_combiner)
encoder_oe = preprocessing.OrdinalEncoder().set_output(transform="pandas")
target_preprocessor = preprocessing.LabelEncoder()

In [31]:
X, y = df.drop(columns=TARGET), df[TARGET]
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size=0.1, random_state=42
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6338, 19), (705, 19), (6338, 1), (705, 1))

In [32]:
# One-hot encoding
X_ohe__train = X_train[CAT_COLS_OHE]
encoder_ohe.fit(X_ohe__train)
X_ohe_trans__train = encoder_ohe.transform(X_ohe__train)
X_ohe_trans_df__train: pd.DataFrame = pd.DataFrame(
    X_ohe_trans__train.toarray(), columns=encoder_ohe.get_feature_names_out()
)

print(X_ohe_trans__train.shape)

# Ordinal Encoding
X_oe__train = X_train[CAT_COLS_LE]
encoder_oe.fit(X_oe__train)
X_oe_trans__train = encoder_oe.transform(X_oe__train)
print(X_oe_trans__train.shape)

# Scale
X_scale__train = X_train[NUM_COLS]
scaler.fit(X_scale__train)
X_scale_trans__train = scaler.transform(X_scale__train)
print(X_scale_trans__train.shape)

X_to_train = pd.concat(
    [
        X_ohe_trans_df__train.reset_index(drop=True),
        X_oe_trans__train.reset_index(drop=True),
        X_scale_trans__train.reset_index(drop=True),
    ],
    axis=1,
)
print(X_to_train.shape)


# ----------------------------------------------- #
X_ohe__test = X_test[CAT_COLS_OHE]
X_ohe_trans__test = encoder_ohe.transform(X_ohe__test)
X_ohe_trans__test: pd.DataFrame = pd.DataFrame(
    X_ohe_trans__test.toarray(), columns=encoder_ohe.get_feature_names_out()
)

X_oe__test = X_test[CAT_COLS_LE]
X_oe_trans__test = encoder_oe.transform(X_oe__test)

X_scale__test = X_test[NUM_COLS]
scaler.fit(X_scale__test)
X_scale_trans__test = scaler.transform(X_scale__test)

X_to_test = pd.concat(
    [
        X_ohe_trans__test.reset_index(drop=True),
        X_oe_trans__test.reset_index(drop=True),
        X_scale_trans__test.reset_index(drop=True),
    ],
    axis=1,
)

(6338, 10)
(6338, 13)
(6338, 3)
(6338, 26)


In [10]:
X_train.shape

(6987, 26)

In [11]:
y_train.shape

(6338,)

In [7]:
target_preprocessor.fit(y_train)
y_train = target_preprocessor.transform(y_train)
y_test = target_preprocessor.transform(y_test)

c:\Users\ishan\Desktop\programs\pythonfiles\churninator\.venv\lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\ishan\Desktop\programs\pythonfiles\churninator\.venv\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\ishan\Desktop\programs\pythonfiles\churninator\.venv\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [9]:
def run_experiments(
    models, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, seed=SEED
) -> pd.DataFrame:
    results = dict()
    for name, model in models:
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = metrics.accuracy_score(y_test, predictions)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            y_test, predictions, average="weighted"
        )
        print(f"{name} --> {fscore}")
        results[name] = (accuracy, precision, recall, fscore)
    return pd.DataFrame.from_dict(results)


models = [
    (
        "dummy_classifier",
        dummy.DummyClassifier(random_state=SEED, strategy="most_frequent"),
    ),
    ("k_nearest_neighbors", neighbors.KNeighborsClassifier()),
    (
        "logistic_regression",
        linear_model.LogisticRegression(
            random_state=SEED, solver="liblinear", class_weight="balanced"
        ),
    ),
    ("support_vector_machines", svm.SVC(random_state=SEED, kernel="rbf")),
    ("random_forest", ensemble.RandomForestClassifier(random_state=SEED)),
    ("gradient_boosting", ensemble.GradientBoostingClassifier(random_state=SEED)),
    ("decision_tree", tree.DecisionTreeClassifier(random_state=SEED)),
    ("adaboost", ensemble.AdaBoostClassifier()),
    (
        "voting",
        ensemble.VotingClassifier(
            estimators=[
                ("gbc", ensemble.GradientBoostingClassifier()),
                ("lr", linear_model.LogisticRegression()),
                ("abc", ensemble.AdaBoostClassifier()),
            ],
            voting="soft",
        ),
    ),
]

results = run_experiments(models)
results.columns = ["accuracy", "precision", "recall", "fscore"]
results = results.sort_values(by=["fscore"], ascending=False)
results

ValueError: Found input variables with inconsistent numbers of samples: [6987, 6338]